# Molecular Dynamics Simulation of Alanine Dipepdite 

In [1]:
import mdtraj as md
from mdtraj.reporters import HDF5Reporter

import nglview as nv
import numpy as np

import openmm as mm
from openmm import app
from openmm import unit

from sys import stdout

## Load alanine dipeptide from file and visualize it

In [2]:
traj = md.load("./alanine-dipeptide.pdb")
topology = traj.topology
print(traj)
print(topology)

<mdtraj.Trajectory with 1 frames, 22 atoms, 3 residues, without unitcells>
<mdtraj.Topology with 1 chains, 3 residues, 22 atoms, 21 bonds>


In [3]:
nv.show_mdtraj(traj)

NGLWidget()

## Prepare for simulation with OpenMM

In [4]:
peptide = app.PDBFile("./alanine-dipeptide.pdb")
modeller = app.Modeller(peptide.topology, peptide.positions)

In [5]:
forcefield = app.ForceField("amber14-all.xml", "amber14/tip3p.xml")

### Solvate

In [6]:
clearance = 14*unit.angstroms
max_size = max(max((pos[i] for pos in modeller.positions))-min((pos[i] for pos in modeller.positions)) for i in range(3))
vectors = mm.Vec3(1.0, 0, 0), mm.Vec3(1.0/3.0, 2.0*np.sqrt(2.0)/3.0,0.0), mm.Vec3(-1.0/3.0, np.sqrt(2.0)/3.0, np.sqrt(6.0)/3.0)
box_vectors = [(max_size + clearance)*v for v in vectors]

modeller.addSolvent(forcefield, model='tip3p', boxVectors = box_vectors, neutralize=True)

In [8]:
max_size

Quantity(value=0.7648, unit=nanometer)

In [9]:
# Verify that water and ions were added
n_waters = 0
ions = []
for chain in modeller.topology.chains():
    for residue in chain.residues():
        if residue.name=='HOH':
            n_waters += 1
        if len(list(residue.atoms()))==1:
            ions += [atom.name for atom in residue.atoms()]            

print('n_waters: {}'.format(n_waters))
print('ions: {}'.format(ions))

n_waters: 228
ions: []


### Create System

In [16]:
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.PME, nonbondedCutoff=0.8*unit.nanometer,
                                 constraints=app.HBonds, rigidWater=True)

for force_index in range(system.getNumForces()):
    force = system.getForce(force_index)
    if isinstance(force, mm.NonbondedForce):
        nonbondedForce = force

nonbondedForce.setUseDispersionCorrection(True)
nonbondedForce.setEwaldErrorTolerance(1.0e-5)
nonbondedForce.setUseSwitchingFunction(True)
nonbondedForce.setSwitchingDistance(0.6*unit.nanometer)

In [17]:
platform = mm.Platform.getPlatformByName('CUDA')

integration_timestep = 2.0*unit.femtoseconds
temperature = 300.0*unit.kelvin

integrator = mm.LangevinIntegrator(temperature, 1.0/unit.picosecond, integration_timestep)

In [18]:
simulation = app.Simulation(modeller.topology, system, integrator, platform)

In [19]:
simulation.context.setPositions(modeller.positions)

### Energy Minimization

In [20]:
# Check the potential energy before minimization
initial_state = simulation.context.getState(getEnergy=True)
initial_state.getPotentialEnergy()

Quantity(value=2809160.2047586255, unit=kilojoule/mole)

In [21]:
simulation.minimizeEnergy()

In [22]:
# Check the potential energy after minimization
minimized_state = simulation.context.getState(getEnergy=True)
minimized_state.getPotentialEnergy()

Quantity(value=-11404.998366374566, unit=kilojoule/mole)

In [23]:
# Writing out the minimized system
minimized_positions = simulation.context.getState(getPositions=True).getPositions()
app.PDBFile.writeFile(simulation.topology, minimized_positions, open('initial.pdb', 'w'))

### Visualized Mimimized system

In [24]:
view = nv.show_structure_file("initial.pdb")

view.representations = [
                {"type": "ball+stick", "params": {
                    "sele": "not ( water or ion )" # Show ligand
                }}
            ]

view.add_licorice(selection="(not protein)")
view

NGLWidget()

In [25]:
simulation.context.setVelocitiesToTemperature(temperature)

In [27]:
simulation_time = 100000.0*unit.picoseconds
reporting_time = 200.0*unit.picoseconds
saving_time = 10.0*unit.picoseconds

simulation_steps = int(simulation_time/integration_timestep)
reporting_steps = int(reporting_time/integration_timestep)
saving_steps = int(saving_time/integration_timestep)

print(f"{simulation_steps} steps")
print(f"{reporting_steps} reporting steps")
print(f"{saving_steps} saving steps")

50000000 steps
100000 reporting steps
5000 saving steps


In [28]:
states_reporter = app.StateDataReporter(stdout, reporting_steps, step=True, potentialEnergy=True, temperature=True,
                                    progress=True, totalSteps=simulation_steps, speed=True, remainingTime=True)
traj_reporter = HDF5Reporter('traj.h5', saving_steps, coordinates=True, time=True, cell=True, potentialEnergy=True,
                             kineticEnergy=True, temperature=True)
checkpoint_reporter = app.CheckpointReporter("./checkpoint.chk", 500)
simulation.reporters+=[states_reporter, traj_reporter, checkpoint_reporter]

In [29]:
simulation.step(simulation_steps)

#"Progress (%)","Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)","Time Remaining"
0.2%,100000,-9141.474928874566,300.0152302440741,0,--
0.4%,200000,-8929.092116374566,302.3619932751897,615,3:53:12
0.6%,300000,-9189.599928874566,292.37447585599455,618,3:51:31
0.8%,400000,-8901.904616374566,305.97253845811025,618,3:51:00
1.0%,500000,-8992.560866374566,294.0279741369378,618,3:50:43
1.2%,600000,-9081.732741374566,294.45531053749505,619,3:50:00
1.4%,700000,-9017.537428874566,292.19536132267933,618,3:49:37
1.6%,800000,-9140.756178874566,287.4047644735468,618,3:49:16
1.8%,900000,-9068.420241374566,290.7872409531826,617,3:49:22
2.0%,1000000,-9096.138991374566,296.7791248529979,614,3:49:50
2.2%,1100000,-8965.607741374566,295.4439307566643,612,3:50:08
2.4%,1200000,-8968.154616374566,317.342753718696,610,3:50:14
2.6%,1300000,-9207.396803874566,310.1187918659765,609,3:50:15
2.8%,1400000,-8958.240553874566,302.6322091031361,609,3:50:00
3.0%,1500000,-9100.373366374566,312.53266

In [30]:
traj_reporter.close()

## Visualize simulation

In [33]:
traj = md.load("traj.h5")
view = nv.show_mdtraj(traj)
view.representations = [
                {"type": "ball+stick", "params": {
                    "sele": "not ( water or ion )" 
                }}
            ]

#view.add_licorice(selection="(not protein)")
view

NGLWidget(max_frame=9999)